<a href="https://colab.research.google.com/github/darshilvekaria/Darshil_Vekaria_GEE/blob/main/MSAVI2_Task_Python_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



                                                                Task Assigned
**To create a Time Series graph of MSAVI 2 index for the three locations 
and analyse the outputs for the study period of January,2020 to June,2020**

# **WORKFLOW**


1) To install necessary packages, import it, Authenticate to the Earth Engine   servers and Initialize the API.

2) To display Earth Engine image tiles on a folium map and to add basemap to it.

3) Locate all the three places.

4) To load sentinel-2 data and write a cloud mask function for it.

5) To write a function to calculate MSAVI2 and add it as a band in the image.

6) Prepare Time Series graph of MSAVI 2 index for place-1 and analyze it.

7) Prepare Time Series graph of MSAVI 2 index for place-2 and analyze it.

8) Prepare Time Series graph of MSAVI 2 index for place-3 and analyze it.

9) Conclusion.


**STEP-1:** To install necessary packages, import it, Authenticate to the Earth Engine   servers and Initialize the API.

In [ ]:
#Installing necessary packages
!pip install rasterio
!pip install ipygee
!pip install tslearn
!pip install earthengine-api
!pip install geopandas

     |████████████████████████████████| 19.3MB 1.8MB/s 
     |████████████████████████████████| 3.3MB 8.6MB/s 
     |████████████████████████████████| 133kB 38.2MB/s 
     |████████████████████████████████| 81kB 5.4MB/s 
     |████████████████████████████████| 225kB 29.6MB/s 
  Created wheel for ipygee: filename=ipygee-0.0.14-cp37-none-any.whl size=39896 sha256=c02dfe5ac11d3a73c2423a9de01d0c61e7262b90b23b798f9d292490f35a998d
  Stored in directory: /root/.cache/pip/wheels/57/f1/a2/ce9509a072ede592c0c9239ba66a1e9bb8af37bab3ca72fabc
  Created wheel for geetools: filename=geetools-0.6.11-cp37-none-any.whl size=90218 sha256=6f7e4a40eefea859e678b1872cc27c6c54dfadaee7fcb1422e4fb64d664c6564
  Stored in directory: /root/.cache/pip/wheels/37/b9/b9/cc9a741a0c7c7bb9fe93949a4ce84356f52e710dd7cba783e1
  Created wheel for pyshp: filename=pyshp-2.1.3-cp37-none-any.whl size=37325 sha256=2c2eef0382db13c5ef300866471d17820ffceb64838dc18b3cf82545caaeaff5
  Stored in directory: /root/.cache/pip/wheels/76/2b

In [ ]:
# Importing necessary packages
import ee
import folium
from folium import plugins
from IPython.display import Image
import geopandas as gpd
import json
print(folium.__version__)
from ipygee import*
import math
import pandas as pd

#  Authenticate to the Earth Engine servers and Initialize the API
ee.Authenticate()
ee.Initialize()

0.8.3
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=EvyfweA-cdHGzvEPic2W69bnkCR257WgwJiBk0jfdhU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7sVCu08DloMzxzsUb1FfTJUdjYisUKAunyvJvjkbHGg_vMF57ef4A

Successfully saved authorization token.


**STEP-2:** To display Earth Engine image tiles on a folium map and to add basemap to it.

In [87]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [88]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

**STEP-3:** Locate all the three places.

In [89]:
# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[30.634, 75.45], zoom_start=16, height=500)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)


# Coordinates of Location-1
Place_1 = ee.Geometry.Polygon(coords = [
  [[75.45157492160797, 30.634404129904425],
  [75.4524278640747, 30.63372099804389],
  [75.45236885547638, 30.634399514164798],
  [75.45157492160797, 30.634404129904425]]],
                               proj= 'EPSG:4326',
                               geodesic = True,
                               maxError= 1.,
                               evenOdd = False)
my_map.add_ee_layer(Place_1, {}, 'Place_1')


# Coordinates of Location-2
Place_2 = ee.Geometry.Polygon(coords = [
  [[85.85622847080231, 26.09425078918021],
  [85.85590660572052, 26.093581136401006], 
  [85.85663080215454, 26.09337879451938 ], 
  [85.85686147212981, 26.094009907326967],
  [85.85622847080231, 26.09425078918021]]],
                               proj= 'EPSG:4326',
                               geodesic = True,
                               maxError= 1.,
                               evenOdd = False)
my_map.add_ee_layer(Place_2, {}, 'Place_2')


# Coordinates of Location-3
Place_3 = ee.Geometry.Polygon(coords = [
  [[78.66571158170699, 17.66869116558751],
  [78.6662346124649, 17.6686911655875],
  [78.6662346124649, 17.66929686130703],
  [78.66571158170699, 17.66929686130703],
  [78.66571158170699, 17.66869116558751]]],
                               proj= 'EPSG:4326',
                               geodesic = True,
                               maxError= 1.,
                               evenOdd = False)
my_map.add_ee_layer(Place_3, {}, 'Place_3')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
display(my_map)

**STEP:4** To load sentinel-2 data and write a cloud mask function for it.

In [90]:
# Load Sentinel-2 Level2A Surface Reflectance imagery, which is Bottom-Of-Atmosphere (BOA) corrected reflectance product.
S2 = ee.ImageCollection("COPERNICUS/S2_SR");

#Writing a function to mask out the cloud cover
def maskS2clouds(image):
  qa = image.select('QA60')

  #Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  #Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).select("B.*").copyProperties(image, ["system:time_start"])


**STEP-5:** To write a function to calculate MSAVI2 and add it as a band in the image.

In [91]:
def addMSAVI2(image):
  MSAVI2 = image.expression(
  '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED)) ) / 2', 
  {
    'NIR': image.select('B8'), 
    'RED': image.select('B4')
  }).rename('MSAVI2')
  return image.addBands(MSAVI2);

**STEP-6:** Prepare Time Series graph of MSAVI 2 index for place-1 and analyze it.

In [92]:
# Filtering data for place-1
S2_place_1 = S2\
  .filterDate('2020-01-01', '2020-07-01').filterBounds(Place_1)
  # .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
  


# .............To apply a function to every Image in an ImageCollection, .map() is useful...............

# Apply cloudmask function(maskS2clouds) to image collection
S2_place_1 = S2_place_1.map(maskS2clouds)

# Add MSAVI2 band to image collection
S2_place_1 = S2_place_1.map(addMSAVI2)


# Selecting MSAVI2 band from the image collection to plot time-series map
S2_place_1 = S2_place_1.select(['MSAVI2'])

# Plot time-series map of MSAVI2
msavi_2_place_1 = chart.Image.series(**{'imageCollection': S2_place_1,
'region': Place_1,
'reducer': ee.Reducer.mean(),
'scale': 10,
'xProperty': 'system:time_start',
'seriesProperty': 'system:index'})

msavi_2_place_1.renderWidget(width='70%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [93]:
# Filtering data for place-2
S2_place_2 = S2\
  .filterDate('2020-01-01', '2020-07-01').filterBounds(Place_2)
  # .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
 


# .............To apply a function to every Image in an ImageCollection, .map() is useful...............

# Apply cloudmask function(maskS2clouds) to image collection
S2_place_2 = S2_place_2.map(maskS2clouds)

# Add MSAVI2 band to image collection
S2_place_2 = S2_place_2.map(addMSAVI2)


# Selecting MSAVI2 band from the image collection to plot time-series map
S2_place_2 = S2_place_2.select(['MSAVI2'])
print('S2_place_2')
# Plot time-series map of MSAVI2
msavi_2_place_2 = chart.Image.series(**{'imageCollection': S2_place_2,
'region': Place_2,
'reducer': ee.Reducer.mean(),
'scale': 10,
'xProperty': 'system:time_start',
'seriesProperty': 'system:index'})

msavi_2_place_2.renderWidget(width='70%')

S2_place_2


HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [94]:
# Filtering data for place-3
S2_place_3 = S2\
.filterDate('2020-01-01', '2020-07-01')\
.filterBounds(Place_3)
# .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
  


# .............To apply a function to every Image in an ImageCollection, .map() is useful...............

# Apply cloudmask function(maskS2clouds) to image collection
S2_place_3 = S2_place_3.map(maskS2clouds)

# Add MSAVI2 band to image collection
S2_place_3 = S2_place_3.map(addMSAVI2)


# Selecting MSAVI2 band from the image collection to plot time-series map
S2_place_3 = S2_place_3.select(['MSAVI2'])

# Plot time-series map of MSAVI2
msavi_2_place_3 = chart.Image.series(**{'imageCollection': S2_place_3,
'region': Place_3,
'reducer': ee.Reducer.mean(),
'scale': 10,
'xProperty': 'system:time_start',
'seriesProperty': 'system:index'})

msavi_2_place_3.renderWidget(width='70%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

**Step-9:** Conclusion
                                        
1) **For location 1**,(One of the corner of Farm,adjacent to road, near Lakha, Punjab)

Crop was fully matured (Ripening stage) in the start and mid of January (0.86). At the end of January, harvesting of crops may have started (0.6) and at the start of April, land was barren with no crops (0.1). Along with this other crop may be sown at the start of April and it may have grown to reach the maturity stage in mid of May, but then it was harvested in June before monsoon arives.


---



2) **For location 2**,(One of the corner of Farm, near Taralahi, Bihar)   
   
There were crops in the farmland from January to February (0.75 to 0.93
.With the start of March, reduction in greenery (crops) started and it kept on decreasing till May (lowest 0.2) and it was not possible get MSAVI2 values for all Sentinel-2 images in June, because of cloud cover.


---


   
3) **For location 3**,(Farmland, near potharam, Telangana) 
 
**Two scenerios**:

**a)** When Images were selected for cloud cover of less than 10%:
 
From January to March, there was greenery (above 0.73) and with the arrival of April month, value starts decreasing and it reached 0.3 at the end of May month indicating barren land. Again like location-2 scenerio, it was not possible get MSAVI2 values in June, because of cloud cover.

**b)** When image were not filtered for cloud cover percentage:
  
There were two sudden drop seen in MSAVI2 value on 9th January and 13th May. Negative MSAVI2 value was obtained on 13th May. This shows that cloud cover do affect the MSAVI2 values
  
Overall, for location 2 and location 3, almost similar trend was seen (decreasing trend of MSAVI2 values), whereas for location 1, there was change in the trend. Within short term period of April to mid of May,
values of MSAVI2 increased from 0.1 to 0.83.
